## Analyse de sentiments avec l'algorithme Vader et le Framework Spacy en Python

In [27]:
import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [28]:
df = pd.read_csv('analyse_sentiment_facebook.csv')

In [29]:
df.head()

,Avis
0,Une autre matinée Twesswiss pour les facebooke...
1,Complètement MALADEAchat Local Haut-Richelieu ...
2,· t1t5 35S71ot5pglo8ct1onams2mo0ar77ebtdre95...
3,"Durant cette pause forcée de Facebook, je me d..."
4,· t1t1335S71 t5pglo8ma1rnsms 2o0ar77e0td1995...


In [30]:
model = SentimentIntensityAnalyzer()

In [31]:
def get_sentiment(text):
    scores = model.polarity_scores(text)
    return scores.get('compound')

In [32]:
%time df['Sentiment'] = np.vectorize(get_sentiment)(df.Avis)

Wall time: 226 ms


In [33]:
df.head()

,Avis,Sentiment
0,Une autre matinée Twesswiss pour les facebooke...,-0.6876
1,Complètement MALADEAchat Local Haut-Richelieu ...,0.0000
2,· t1t5 35S71ot5pglo8ct1onams2mo0ar77ebtdre95...,0.0000
3,"Durant cette pause forcée de Facebook, je me d...",0.0000
4,· t1t1335S71 t5pglo8ma1rnsms 2o0ar77e0td1995...,0.0000


In [34]:
import spacy
import re

In [43]:
nlp = spacy.load('en_core_web_sm')

def get_entities(text):
    doc = nlp(text)
    list_ = []
    for ent in doc.ents:
        if ent.label_.lower() not in ('time', 'money', 'date'):
            temp_  = re.sub('[^a-z0-9]+', '_', ent.text.lower())
            list_.append(' '.join([temp_, ent.label_.upper()]))
    return ''.join(list_)

In [44]:
%time df['entities'] = np.vectorize(get_entities)(df.Avis)

Wall time: 8.85 s


## Sentiments négatifs des utilisateurs sur les réseaux sociaux 

In [71]:
sentiment_negative = df[df['Sentiment']<0]
sentiment_negative

,Avis,Sentiment,entities
0,Une autre matinée Twesswiss pour les facebooke...,-0.6876,twesswiss GPEpannefacebook131311 LAWvoir PERSO...
8,"#pannefacebookBonsoir, suite à une panne génér...",-0.5994,_nouveaux PERSONvoir PERSON
13,CE SOIR TOUT LE MONDE EST CONNECTÉ !Nous avons...,-0.4376,lu PERSONqui_va_d_sormais ORGvoir PERSON
16,LE CHAOS EST TERMINÉ! Vous êtes toujours là?On...,-0.7620,le_chaos_est ORGl_on PERSON_a_va PERSONremis_d...
22,La danse saturno-uranienne aura encore fait p...,-0.5255,fait_parler ORGd_elle GPEdans NORPl_lectricit_...
...,...,...,...
789,"De son côté, le BGP — pour Border Gateway Prot...",-0.7351,bgp ORGatteindre_l_adresse PERSONbgp ORGdu_poi...
791,"En 6heures, il y a certainement eu la possibil...",-0.3182,eu_la_possibilit_ PERSONbien PERSONdis_a PERSO...
802,"Black Friday Amazon : les produits Echo, Kindl...",-0.1779,amazon ORGecho GPEkindle_et_fire_tv ORG
815,Kawasaki prend d’assaut les motos électriques ...,-0.4588,kawasaki GPEd_assaut NORP


### Pourcentage des sentiments négatives

In [77]:
pourcentage_sentiment_negative = (len(sentiment_negative)/len(df))*100
print("LES UTILISATEURS SONT PESSIMISTES A {:.2f} %".format(pourcentage_sentiment_negative))

LES UTILISATEURS SONT PESSIMISTES A 8.07 %


## Sentiments positifs des utilisateurs sur les réseaux sociaux

In [78]:
sentiment_positif = df[df['Sentiment']>0]
sentiment_positif

,Avis,Sentiment,entities
5,Grace à la panne mondiale que subit actuelleme...,0.5574,10 CARDINALd_ailleurs PERSON2 CARDINAL_ CARDINAL
30,#Pannefacebook#webgospel#webdaysedition2 Selon...,0.3400,pannefacebook_webgospel_webdaysedition2_selon ...
31,Happiness FM: Franck Mansuy Formateur - Anima...,0.5574,franck_mansuy PERSONformateur ORG
33,La panne géante des réseaux sociaux vous a cer...,0.8020,jouer_des_jeux_de_soci_t_ ORGdans NORPqui GPEc...
38,#IdéesReçues #CasSociaux #PanneFacebook Le myt...,0.4738,pannefacebook_le PRODUCTmais_la_panne PERSONal...
...,...,...,...
738,La pépite française EfficientIP annonce une so...,0.7096,trafic_qui GPEfournisseur_de_solutions_de_s_cu...
740,"Les podcast Radio DSI, Radio DCmag et les micr...",0.7003,radio_dsi ORGradio_dcmag ORGdatacenter_magazin...
763,Facebook indique ainsi que sa panne aurait eu ...,0.2263,aurait_eu_pour PERSONdans NORPqui ORGun_mauvai...
790,Ils ont perdu environ 5-6% en bourse pendant q...,0.4939,5_6_ PERCENTc_tait GPEsur_instagram_et_faceboo...


### Pourcentage des sentiments positifs

In [79]:
pourcentage_sentiment_positif = (len(sentiment_positif)/len(df))*100
print("LES UTILISATEURS SONT PESSIMISTES A {:.2f} %".format(pourcentage_sentiment_positif))

LES UTILISATEURS SONT PESSIMISTES A 10.60 %


## Sentiments neutres des utilisateurs sur les réseaux sociaux

In [83]:
sentiment_neutre = df[df['Sentiment']==0]
sentiment_neutre

,Avis,Sentiment,entities
1,Complètement MALADEAchat Local Haut-Richelieu ...,0.0,
2,· t1t5 35S71ot5pglo8ct1onams2mo0ar77ebtdre95...,0.0,partag_ GPEgroupe ORG
3,"Durant cette pause forcée de Facebook, je me d...",0.0,de_facebook PERSONpannefacebook ORG
4,· t1t1335S71 t5pglo8ma1rnsms 2o0ar77e0td1995...,0.0,2o0ar77e0td1995 CARDINALpartag_ GPE
6,"NaNo DanielAsso VDR ""Voyager dos à la route"" (...",0.0,nano PERSON
...,...,...,...
825,Stéphane Richard démissionne de son poste de P...,0.0,st_phane_richard PERSONd_missionne_de_son PERS...
826,"iPhone 13, Apple Watch Series 6, AirPods Pro… ...",0.0,13 CARDINALapple_watch_series ORG6 CARDINALair...
827,Rejoignez la discussion,0.0,rejoignez NORP
828,À propos • Mentions légales • Contacts • Archives,0.0,_mentions ORG


## Pourcentage des sentiments neutres

In [84]:
pourcentage_sentiment_neutre = (len(sentiment_neutre)/len(df))*100
print("LES UTILISATEURS SONT PESSIMISTES A {:.2f} %".format(pourcentage_sentiment_neutre))

LES UTILISATEURS SONT PESSIMISTES A 81.33 %


###### De ces constats, nous pouvons conclure que les utilisateurs sont dans une position de mefiance, c'est à dire 
###### le moment de se remttre en question sur dependances de facebook et ses filliale.

##  Perceptive

#### On pourrait aller plus encore en servant des enties: par nous pouvons chercher des pays plus 
#### de mention ou le contraire ou encore parmi les négative combien de fois le fondateur cité ..........

In [134]:
import re

france_neg = []

for w in df[df['Sentiment']<0].iloc[:, -1]: 
    if 'france' in w.lower():
        france_neg.append(w)
    
print('Sur {} négatif(s) les français ont négativement parlés sur facebook {} fois'.format(
    len(df[df['Sentiment']<0].iloc[:, -1]),
    len(france_neg)
))

Sur 67 négatif les français ont négativement sur facebook de 1


In [136]:
france_pos = []

for w in df[df['Sentiment']>0].iloc[:, -1]: 
    if 'france' in w.lower():
        france_pos.append(w)
    
france_pos

print('Sur {} positif(s) les français ont positifivement parlés sur facebook {} fois'.format(
    len(df[df['Sentiment']>0].iloc[:, -1]),
    len(france_pos)
))

Sur 88 positif(s) les français ont positifivement parlés sur facebook 4 fois


In [138]:
france_neutre = []

for w in df[df['Sentiment']>0].iloc[:, -1]: 
    if 'france' in w.lower():
        france_neutre.append(w)
    
france_neutre

print('Sur {} neutre(s) les français ont  parlés de façon sur facebook {} fois'.format(
    len(df[df['Sentiment']==0].iloc[:, -1]),
    len(france_neutre)
))

Sur 675 neutre(s) les français ont  parlés de façon sur facebook 4 fois


In [139]:
print('Sur {} avis les français ont  réagi {} fois'.format(
    len(df),
    len(france_neutre)+len(france_neg) + len(france_pos)
))

Sur 830 avis les français ont  réagi 9 fois
